In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
import os
import subprocess 
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.source import AreaSource
from openquake.hazardlib.const import TRT
from openquake.hazardlib.geo import NodalPlane
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.mfd import TruncatedGRMFD

In [ ]:
try:
    print_log
except: 
    print_log = False

In [ ]:
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
fin = open(data_path,'rb') 
sources = pickle.load(fin)
fin.close()

In [ ]:
src = sources[area_source_ids_list[0]]

In [ ]:
nodal_plane_distribution = PMF(data=[(1.0, NodalPlane(strike=0, dip=90, rake=0))]) 
hypocenter_distribution = PMF([(1.0, 5.)])

src.m_max = 7.0 
src.tectonic_region_type = TRT.ACTIVE_SHALLOW_CRUST
src.rupture_mesh_spacing = 5.
src.magnitude_scaling_relationship = WC1994() 
src.rupture_aspect_ratio = 1.0 
src.temporal_occurrence_model = PoissonTOM(1.0) 
src.nodal_plane_distribution = nodal_plane_distribution
src.hypocenter_distribution = hypocenter_distribution
src.upper_seismogenic_depth = 0. 
src.lower_seismogenic_depth = 20.
src.area_discretization = 10.

In [ ]:
bin_width = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
min_mag = float(metys.g_prj.mod[model_key]['mfd_min_magnitude'])

src.mfd = TruncatedGRMFD(min_mag = min_mag,
                         max_mag = src.m_max,
                         bin_width = bin_width,
                         a_val = src.a_gr,
                         b_val = src.b_gr)

In [ ]:
sources[area_source_ids_list[0]] = src
output_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)  
fou = open(output_path,'wb') 
pickle.dump(sources, fou)
fou.close()